[View in Colaboratory](https://colab.research.google.com/github/Santosh-Gupta/BookCrossingsRecommender/blob/master/BookCrossingsDataAnalysis.ipynb)

Import Stuff, set connection to Google Drive

In [0]:
import pandas as pd

!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile
import os
import sys

import random
import numpy as np
import pandas as pd

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Download BookCrossings data from Google Drive

In [2]:
dl_id = input("Enter Gdrive file ID for Book Crossings data file: ") # 1NIYYr2QZ9kSjavcpou08cYkyFJ83JpMh

# DOWNLOAD ZIP
print ("Downloading  file")
myzip = drive.CreateFile({'id': dl_id})
myzip.GetContentFile('data.csv')

print( os.listdir() )

Enter Gdrive file ID for Book Crossings data file: 1NIYYr2QZ9kSjavcpou08cYkyFJ83JpMh
['datalab', '.cache', '.local', '.ipython', '.config', 'data.csv', '.forever']


Open data and convert to Pandas dataframe object. Look at number of rows (data points)


In [3]:
with open('data.csv', 'rb') as f:
  text = f.read()
  
csvfile = pd.read_csv('data.csv',  encoding = "ISO-8859-1", error_bad_lines=False, delimiter=';')
print(csvfile.head())
csvfile.shape[0]

   User-ID        ISBN  Book-Rating
0   276725  034545104X            0
1   276726  0155061224            5
2   276727  0446520802            0
3   276729  052165615X            3
4   276729  0521795028            6


1149780

Columns must be renamed for Pandas compatability ( ie pd.Book-Rating will not work but pd.BookRating will)



In [0]:
csvfile.rename(columns={'Book-Rating': 'BookRating', 'User-ID': 'UserID'}, inplace=True)

I decided to filter out results where the rating was below a 6. Since these vectors are going to be used for book reccomendations, I decided to only train on data points where the books resulted in an enjoyable experience for the user. 

In [5]:
csvfile6plus = csvfile[csvfile.BookRating >= 6] 
print(csvfile6plus.head())
csvfile6plus.shape[0] #check how many data points we have left

   UserID        ISBN  BookRating
4  276729  0521795028           6
6  276736  3257224281           8
7  276737  0600570967           6
8  276744  038550120X           7
9  276745   342310538          10


363268

Look at statics about the books. In the stats below, count is the total number of books and mean is the average amount of times a book appears in the data. Min is the occurance for the lowest occuring book, or multiple books if there are more than one book that has the same number for lowest occurance.

In [6]:
reviews_per_user45 = csvfile6plus.groupby( 'ISBN' ).UserID.apply( lambda x: len( x ))
reviews_per_user45.describe() 

count    159323.000000
mean          2.280073
std           6.418580
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         663.000000
Name: UserID, dtype: float64

Filter out books that appear less than two times because books with lower instances will not get too many opportunities to train their embedding during each epoch. For example, a book that only occurs once in the data will only train once per epoch (assuming that each user appears at least twice in the data). Also, since these embeddings will be used for reccomendation, it would be ideal to have each book be reviewed by multiple users. But since I am not working with too much data, I decided to filter out books with only one occurance. 

In [0]:
over2users = csvfile6plus.groupby('ISBN').filter(lambda x: len(x) >= 2) 

Look at statics about the users. In the stats below, count is the total number of users and mean is the average amount of times a user appears in the data. Min is the occurance for the lowest occuring user, or multiple users if there are more than one user that has the same number for lowest occurance.

In [8]:
checkUsers = over2users.groupby( 'UserID' ).UserID.apply( lambda x: len( x ))
checkUsers.describe() 

count    56666.000000
mean         4.431846
std         28.388489
min          1.000000
25%          1.000000
50%          1.000000
75%          3.000000
max       5172.000000
Name: UserID, dtype: float64

It is necessary to have at least two occurances for each user. The word2vec-based algorithm will measure similiarity among all books rated by a user, and so there should be at least two books rated by a particular user. 

In [0]:
over2usersOver2books = csvfile6plus.groupby('UserID').filter(lambda x: len(x) >= 2) 

Check the stats of the books again, in case removing all instances of one user resulted in cases of only one book. 

In [10]:
checkBooks = over2usersOver2books.groupby( 'ISBN' ).ISBN.apply( lambda x: len( x )) 
checkBooks.describe() 

count    147362.000000
mean          2.183989
std           5.663278
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max         518.000000
Name: ISBN, dtype: float64

Filter out cases of only one book again. This resulted in cases where there was only one user again, so the next several books is just going back and forth removing cases of one book and one user. There's better ways to code this. 

In [0]:
over2usersOver2booksPart2 = over2usersOver2books.groupby('ISBN').filter(lambda x: len(x) >= 2)  #Since there are cases of only 1 user. 

In [12]:
checkBooks = over2usersOver2booksPart2.groupby( 'UserID' ).ISBN.apply( lambda x: len( x ))  #check to see if removing cases of 1 user created cases where there is only 1 book review
checkBooks.describe() 

count    26354.000000
mean         8.222092
std         39.183844
min          1.000000
25%          2.000000
50%          3.000000
75%          6.000000
max       4790.000000
Name: ISBN, dtype: float64

In [0]:
over2usersOver2booksPart3 = over2usersOver2booksPart2.groupby('UserID').filter(lambda x: len(x) >= 2) 

In [14]:
checkBooks = over2usersOver2booksPart3.groupby( 'ISBN' ).ISBN.apply( lambda x: len( x ))  #check to see if removing cases of 1 book reviews created cases where there is only 1 user 
checkBooks.describe() 

count    42175.000000
mean         5.042087
std          9.873195
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max        511.000000
Name: ISBN, dtype: float64

In [0]:
over2usersOver2booksPart4 = over2usersOver2booksPart3.groupby('ISBN').filter(lambda x: len(x) >= 2) 

In [16]:
checkBooks = over2usersOver2booksPart4.groupby( 'UserID' ).ISBN.apply( lambda x: len( x ))  
checkBooks.describe() 

count    22317.000000
mean         9.477976
std         41.913647
min          1.000000
25%          2.000000
50%          4.000000
75%          7.000000
max       4697.000000
Name: ISBN, dtype: float64

In [0]:
over2usersOver2booksPart5 = over2usersOver2booksPart4.groupby('UserID').filter(lambda x: len(x) >= 2) 

In [18]:
checkBooks = over2usersOver2booksPart5.groupby( 'ISBN' ).ISBN.apply( lambda x: len( x ))  
checkBooks.describe() 

count    41044.000000
mean         5.150302
std          9.983747
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max        511.000000
Name: ISBN, dtype: float64

In [0]:
over2usersOver2booksPart6 = over2usersOver2booksPart5.groupby('ISBN').filter(lambda x: len(x) >= 2) 

In [20]:
checkBooks = over2usersOver2booksPart6.groupby( 'UserID' ).ISBN.apply( lambda x: len( x ))  
checkBooks.describe() 

count    22185.000000
mean         9.526166
std         42.007800
min          1.000000
25%          2.000000
50%          4.000000
75%          7.000000
max       4692.000000
Name: ISBN, dtype: float64

In [0]:
over2usersOver2booksPart7 = over2usersOver2booksPart6.groupby('UserID').filter(lambda x: len(x) >= 2) 

In [22]:
checkBooks = over2usersOver2booksPart7.groupby( 'ISBN' ).ISBN.apply( lambda x: len( x ))  
checkBooks.describe() 

count    40993.000000
mean         5.155197
std          9.988682
min          1.000000
25%          2.000000
50%          3.000000
75%          5.000000
max        511.000000
Name: ISBN, dtype: float64

In [0]:
over2usersOver2booksPart8 = over2usersOver2booksPart7.groupby('ISBN').filter(lambda x: len(x) >= 2) 

In [24]:
checkBooks = over2usersOver2booksPart8.groupby( 'UserID' ).ISBN.apply( lambda x: len( x ))  
checkBooks.describe() 

count    22174.000000
mean         9.530170
std         42.012737
min          2.000000
25%          2.000000
50%          4.000000
75%          7.000000
max       4691.000000
Name: ISBN, dtype: float64

In [25]:
checkBooks = over2usersOver2booksPart8.groupby( 'ISBN' ).ISBN.apply( lambda x: len( x ))  #Important to note total number of books is 40988
checkBooks.describe() 

count    40988.000000
mean         5.155704
std          9.989186
min          2.000000
25%          2.000000
50%          3.000000
75%          5.000000
max        511.000000
Name: ISBN, dtype: float64

Finally, at least two occurances of each user and each book in the data. Lets check the number of data points we have left. 

In [26]:
over2usersOver2booksPart8.shape[0]

211322

The data will now be sorted by the number of occurances for each book. We will be mapping the ISBN with numbers (EmbedID) from 0 to the number of books , and these numbers will be used for looking of the vector for each book in the embedding and softmax matrices. For now, the data is ordered by book occurance (we will randomize the data later) so that books that occur more frequently in the data will have smaller number for the EmbedID. We are doing this because we will mostly be focusing on books that occur more frequently (and thus whose embeddings had more training) in the analysis, and arranging the EmbedID this way makes it easier. This probably doesn't make sence at the moment, but as you go through this notebook, the traning notebook, and the analysis notebook hopefully it'll be more clear. So if this doesn't quite make sense yet, wait until you have reviewed those notebooks and then come back to this section. 

In [29]:
over2usersOver2booksPart8['count'] = over2usersOver2booksPart8.groupby('ISBN')['ISBN'].transform(pd.Series.value_counts)
over2usersOver2booksPart8.head(30)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,UserID,ISBN,BookRating,count
16,276747,0060517794,9,26
19,276747,0671537458,9,14
20,276747,0679776818,8,14
59,276772,0553572369,7,7
61,276772,3499230933,10,3
77,276786,8437606322,8,2
81,276786,8478442588,6,3
83,276788,0345443683,8,18
84,276788,043935806X,7,176
85,276788,055310666X,10,11


In [33]:
sortedData = over2usersOver2booksPart8.sort_values('count', ascending=False)
sortedData.head(20)

,UserID,ISBN,BookRating,count
549397,131686,0316666343,8,511
468822,112026,0316666343,8,511
990646,238131,0316666343,8,511
1117422,268110,0316666343,7,511
989554,238120,0316666343,7,511
1094935,262795,0316666343,9,511
1074012,256873,0316666343,10,511
367245,88229,0316666343,10,511
151329,34377,0316666343,10,511
222349,52002,0316666343,10,511


In [35]:
sortedData2 = sortedData.drop('count', axis=1)
sortedData2.head(20)

,UserID,ISBN,BookRating
549397,131686,0316666343,8
468822,112026,0316666343,8
990646,238131,0316666343,8
1117422,268110,0316666343,7
989554,238120,0316666343,7
1094935,262795,0316666343,9
1074012,256873,0316666343,10
367245,88229,0316666343,10
151329,34377,0316666343,10
222349,52002,0316666343,10


In [0]:
sortedData2.to_csv('refinedBookCrossingsRatings4.csv') #To make it into a regular dataframe object again
                                                                                                      #There's probably a better way to convert the data into a dataframe object again
rr2 = pd.read_csv( 'refinedBookCrossingsRatings4.csv' )

Map each ISBN to a unique number called the EmbedID. This number will be used to look up a book's embedding from the embedding and softmax matrices. For example, a book with EmbedID of 7 will have it's embedding be the 7th row (or column; I forgot how the matrix is orientated) in the embedding and softmax matrix. 

In [37]:
mapping = {k: v for v, k in enumerate(rr2.ISBN.unique())}  #Give each book a unique ID for embedding matrix and softmax matrix lookup
rr2['EmbedID'] = rr2.ISBN.map(mapping)
rr2.head()

,Unnamed: 0,UserID,ISBN,BookRating,EmbedID
0,549397,131686,0316666343,8,0
1,468822,112026,0316666343,8,0
2,990646,238131,0316666343,8,0
3,1117422,268110,0316666343,7,0
4,989554,238120,0316666343,7,0


Group the data by each User. This format is neccessary for the way I coded the training algorithm. This is for the algorithm to know which books for each data point to perform similiarity on. 

In [38]:
rr3= rr2.groupby(['UserID'], as_index=False, sort=True)

#rr3.head(63)

print(rr3.apply(lambda x: x.head(200)))

              Unnamed: 0  UserID        ISBN  BookRating  EmbedID
0     61670         9598      17  0891075275           6     1504
      120080        9594      17  0425099148           7     6515
1     4507          9607      26  0446310786          10       19
      14487         9608      26  0449005615           9      106
2     43342         9617      39  0671888587           7      732
      96544         9616      39  0553582909           8     4277
3     120281        9628      53  0380715899           9     7320
      160989        9626      53  0156047624          10    20221
4     65067         9632      56  0671623249           7     1698
      160974        9634      56  0679865691           9    17188
5     96204         9667      92  8445071408           7     4186
      120205        9669      92  8445071777          10     6591
      120206        9668      92  8445071769           8     6602
6     40196         9677      99  0446677450          10      611
      9299

This block will add two columns, one is a random number mapped for each user, and the other column is just a random number. These two columns will be used to randomize the order of the users, and the order of the books for each user. This is to prevent the training algorithm from fitting to the order of the data. I like to train on a few few versions of the data with this randomization.  

In [39]:

randomColumn1 = dict()
i=1
for n in random.sample(range(1, rr2['UserID'].max()+1), rr2['UserID'].max()):
  randomColumn1.update({i:n})
  i=i+1
  
#randomColumn2 = dict()
#i=1
#for n in random.sample(range(1, rr2['user_id'].max()+1), rr2['user_id'].max()):
#  randomColumn2.update({i:n})
#  i=i+1


  
rr2['SortIndex1'] = rr2['UserID'].map(randomColumn1)

#rr2['SortIndex2'] = rr2['user_id'].map(randomColumn2)

rr2['SortIndex2'] = np.random.randint(1, 500, rr2.shape[0])
rr2.head()

,Unnamed: 0,UserID,ISBN,BookRating,EmbedID,SortIndex1,SortIndex2
0,549397,131686,0316666343,8,0,274152,140
1,468822,112026,0316666343,8,0,115852,61
2,990646,238131,0316666343,8,0,73252,57
3,1117422,268110,0316666343,7,0,58798,18
4,989554,238120,0316666343,7,0,250700,372


In [40]:
r3 = rr2.sort_values(['SortIndex1', 'SortIndex2']) 

r3.head(50)


,Unnamed: 0,UserID,ISBN,BookRating,EmbedID,SortIndex1,SortIndex2
45369,1082427,259004,0749395990,8,784,22,54
175732,1082450,259004,8388553410,7,24422,22,91
87162,1082415,259004,055356983X,7,3085,22,183
175769,1082363,259004,0146001788,6,24458,22,250
76825,1082357,259004,0140430113,6,2465,22,251
111093,1082426,259004,0747525722,8,6411,22,258
147748,1082425,259004,0743225198,8,14199,22,260
104248,1082422,259004,0676973760,10,5035,22,264
89667,1082391,259004,0451158717,9,3447,22,267
76809,1082412,259004,0553268937,9,2483,22,281


Here I add a new column called 'Headernumber'. This column is used to keep track of data points that belong to a particular user. For the first instance of a particular user in the data, the headernumber is the number of data points for that user, and for the rest of the instances, the headernumber is '-1'. When the training algorithm is going through the data, the headernumber is used to match the right inputs with the right labels. How exactly the Headernumber is used will be explained more clearly in the training algorithm. 

In [41]:
r3['Headernumber'] = np.where(r3['UserID'].duplicated(), -1, 
                         r3.groupby('UserID')['UserID'].transform(len))

r3.head(20)

,Unnamed: 0,UserID,ISBN,BookRating,EmbedID,SortIndex1,SortIndex2,Headernumber
45369,1082427,259004,0749395990,8,784,22,54,17
175732,1082450,259004,8388553410,7,24422,22,91,-1
87162,1082415,259004,055356983X,7,3085,22,183,-1
175769,1082363,259004,0146001788,6,24458,22,250,-1
76825,1082357,259004,0140430113,6,2465,22,251,-1
111093,1082426,259004,0747525722,8,6411,22,258,-1
147748,1082425,259004,0743225198,8,14199,22,260,-1
104248,1082422,259004,0676973760,10,5035,22,264,-1
89667,1082391,259004,0451158717,9,3447,22,267,-1
76809,1082412,259004,0553268937,9,2483,22,281,-1


Reorder the columns

In [42]:
r3 = r3[['Unnamed: 0', 'UserID', 'EmbedID', 'BookRating', 'SortIndex1', 'SortIndex2',  'Headernumber',   'ISBN']]
r3.head(20)

,Unnamed: 0,UserID,EmbedID,BookRating,SortIndex1,SortIndex2,Headernumber,ISBN
45369,1082427,259004,784,8,22,54,17,0749395990
175732,1082450,259004,24422,7,22,91,-1,8388553410
87162,1082415,259004,3085,7,22,183,-1,055356983X
175769,1082363,259004,24458,6,22,250,-1,0146001788
76825,1082357,259004,2465,6,22,251,-1,0140430113
111093,1082426,259004,6411,8,22,258,-1,0747525722
147748,1082425,259004,14199,8,22,260,-1,0743225198
104248,1082422,259004,5035,10,22,264,-1,0676973760
89667,1082391,259004,3447,9,22,267,-1,0451158717
76809,1082412,259004,2483,9,22,281,-1,0553268937


Check how many books there are

In [43]:
r3.loc[r3['EmbedID'].idxmax()]

Unnamed: 0          716284
UserID              173517
EmbedID              40987
BookRating               9
SortIndex1          137783
SortIndex2             208
Headernumber            -1
ISBN            1573245232
Name: 211276, dtype: object

Since the training algorithm uses EmbedID, ISBN is dropped to reduce size of the data file. 

In [44]:
r4 = r3.drop(columns=['ISBN'])
r4.head(20)

,Unnamed: 0,UserID,EmbedID,BookRating,SortIndex1,SortIndex2,Headernumber
45369,1082427,259004,784,8,22,54,17
175732,1082450,259004,24422,7,22,91,-1
87162,1082415,259004,3085,7,22,183,-1
175769,1082363,259004,24458,6,22,250,-1
76825,1082357,259004,2465,6,22,251,-1
111093,1082426,259004,6411,8,22,258,-1
147748,1082425,259004,14199,8,22,260,-1
104248,1082422,259004,5035,10,22,264,-1
89667,1082391,259004,3447,9,22,267,-1
76809,1082412,259004,2483,9,22,281,-1


Save the file

In [0]:
r4.to_csv('BookCrossingDataReadyForTraining6.csv', index=False)

Upload file to Google Drive 

In [0]:
# 2. Create & upload a file text file.
uploadModel = drive.CreateFile()

#zipname = 'RatingRefined'
#zipfolder(zipname, 'checkpoints5')
uploadModel.SetContentFile("BookCrossingDataReadyForTraining6.csv") #name of file located in local disk 
uploadModel.Upload()

Now that we're done with the training data, lets create a data file which holds the EmbedID, ISBN, book title, and the author. First, lets download the BX-Books.csv file. 

In [47]:
dl_id = input("Enter Gdrive file ID for Book titles file: ") # 1kZBM8orAm9OpKLHWuVCp2YqFY-LxaYim

# DOWNLOAD ZIP
print ("Downloading  file")
myzip = drive.CreateFile({'id': dl_id})
myzip.GetContentFile('bookTitles.csv')

print( os.listdir() )

Enter Gdrive file ID for Book titles file: 1kZBM8orAm9OpKLHWuVCp2YqFY-LxaYim
['datalab', 'refinedBookCrossingsRatings4.csv', '.cache', '.local', '.ipython', 'BookCrossingDataReadyForTraining6.csv', '.config', 'refinedBookCrossingsRatings3.csv', 'data.csv', 'bookTitles.csv', '.forever']


Open and convert data to a pandas dataframe

In [48]:
with open('bookTitles.csv', 'rb') as f:
  text = f.read()
bookfile = pd.read_csv('bookTitles.csv',  encoding = "ISO-8859-1", error_bad_lines=False, delimiter=';')
bookfile.head()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:27

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


Create a dictionary for the ISBN and Title

In [49]:
bookDictionary = bookfile.set_index('ISBN').to_dict()['Book-Title']
bookDictionary['0393045218']

'The Mummies of Urumchi'

Create a dictionary for the ISBN and Title

In [50]:
authorDictionary = bookfile.set_index('ISBN').to_dict()['Book-Author']
authorDictionary['0393045218']

'E. J. W. Barber'

Use the dictionary to create columns for the r3 pandas dataframe for the book title and author

In [51]:
r3['Book_Title'] = r3['ISBN'].map(bookDictionary)
r3['Author'] = r3['ISBN'].map(authorDictionary)
r3.head(20)

,Unnamed: 0,UserID,EmbedID,BookRating,SortIndex1,SortIndex2,Headernumber,ISBN,Book_Title,Author
45369,1082427,259004,784,8,22,54,17,0749395990,NaN,NaN
175732,1082450,259004,24422,7,22,91,-1,8388553410,NaN,NaN
87162,1082415,259004,3085,7,22,183,-1,055356983X,Anne Frank's Tales from the Secret Annex,Anne Frank
175769,1082363,259004,24458,6,22,250,-1,0146001788,"The judgement ;: And, In the penal colony (Pen...",Franz Kafka
76825,1082357,259004,2465,6,22,251,-1,0140430113,Jane Eyre (Penguin Classics),Charlotte Bronte
111093,1082426,259004,6411,8,22,258,-1,0747525722,The Tortilla Curtain,Coraghessan T. Boyle
147748,1082425,259004,14199,8,22,260,-1,0743225198,More George W. Bushisms: More of Slate's Accid...,Jacob Weisberg
104248,1082422,259004,5035,10,22,264,-1,0676973760,NaN,NaN
89667,1082391,259004,3447,9,22,267,-1,0451158717,Who's Afraid of Virginia Woolf?,Edward Albee
76809,1082412,259004,2483,9,22,281,-1,0553268937,The Water Is Wide,PAT CONROY


Now we can create dictionary's to look up the book title and author from the EmbedID

In [52]:
bookDictionary2 = r3.set_index('EmbedID').to_dict()['Book_Title']
bookDictionary2[95]

'The Hours: A Novel'

In [53]:
authorDictionary2 = r3.set_index('EmbedID').to_dict()['Author']
authorDictionary2[135]

'Janet Evanovich'

Remove training data

In [0]:
r5 = r3.drop(columns=['Unnamed: 0', 'UserID', 'BookRating', 'SortIndex1', 'SortIndex2', 'Headernumber' ])

Remove duplicates

In [55]:
r5.drop_duplicates()
r5.head()

,EmbedID,ISBN,Book_Title,Author
45369,784,0749395990,NaN,NaN
175732,24422,8388553410,NaN,NaN
87162,3085,055356983X,Anne Frank's Tales from the Secret Annex,Anne Frank
175769,24458,0146001788,"The judgement ;: And, In the penal colony (Pen...",Franz Kafka
76825,2465,0140430113,Jane Eyre (Penguin Classics),Charlotte Bronte


Save and upload file to Google drive

In [0]:
r5.to_csv('BookCrossingBookData3.csv', index=False)

# 2. Create & upload a file text file.
uploadModel = drive.CreateFile()

#zipname = 'RatingRefined'
#zipfolder(zipname, 'checkpoints5')
uploadModel.SetContentFile("BookCrossingBookData3.csv") #name of file located in local disk 
uploadModel.Upload()